In [20]:
train_dir = "/kaggle/input/cellula-task1/Teeth_Dataset/Training"
val_dir = "/kaggle/input/cellula-task1/Teeth_Dataset/Validation"
test_dir = "/kaggle/input/cellula-task1/Teeth_Dataset/Testing"

In [21]:
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras  # Explicitly import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load Data
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')

# Keep only the required 7 classes in test set
allowed_classes = ['CaS', 'CoS', 'Gum', 'MC', 'OC', 'OLP', 'OT']
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', classes=allowed_classes)

# Verify class labels
print("Class Labels:", train_generator.class_indices)

Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Class Labels: {'CaS': 0, 'CoS': 1, 'Gum': 2, 'MC': 3, 'OC': 4, 'OLP': 5, 'OT': 6}


# Lets try model_2 with EfficientNetB3



In [23]:
def build_efficientnet():
    base_model = keras.applications.EfficientNetB3(
        weights="imagenet", include_top=False, input_shape=(224, 224, 3)
    )
    base_model.trainable = False  # Freeze feature extractor

    model_2 = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(7, activation='softmax')
    ])

    model_2.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model_2


In [24]:
efficientnet_model = build_efficientnet()
history_efficientnet = efficientnet_model.fit(train_generator, epochs=20, validation_data=val_generator)




Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


97/97 ━━━━━━━━━━━━━━━━━━━━ 101s 690ms/step - accuracy: 0.1520 - loss: 2.5345 - val_accuracy: 0.1167 - val_loss: 1.9971
Epoch 2/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 334ms/step - accuracy: 0.1463 - loss: 2.3781 - val_accuracy: 0.1751 - val_loss: 2.0041
Epoch 3/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 333ms/step - accuracy: 0.1647 - loss: 2.3407 - val_accuracy: 0.1751 - val_loss: 1.9896
Epoch 4/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 34s 330ms/step - accuracy: 0.1442 - loss: 2.2699 - val_accuracy: 0.1751 - val_loss: 1.9792
Epoch 5/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 333ms/step - accuracy: 0.1453 - loss: 2.2766 - val_accuracy: 0.1751 - val_loss: 1.9695
Epoch 6/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 332ms/step - accuracy: 0.1612 - loss: 2.2763 - val_accuracy: 0.1751 - val_loss: 1.9598
Epoch 7/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 333ms/step - accuracy: 0.1605 - loss: 2.2503 - val_accuracy: 0.1751 - val_loss: 1.9511
Epoch 8/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 331ms/step - accuracy: 0.1571 - loss: 2.2407 - val_accuracy: 0.17

In [29]:
test_loss, test_acc = efficientnet_model.evaluate(test_generator)
print(f"EfficientNetB3 Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.1774 - loss: 1.9095
EfficientNetB3 Test Accuracy: 0.1761, Test Loss: 1.9191


In [30]:
def build_densenet():
    base_model = keras.applications.DenseNet121(
        weights="imagenet", include_top=False, input_shape=(224, 224, 3)
    )
    base_model.trainable = False  # Freeze feature extractor

    model_3 = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(7, activation='softmax')
    ])

    model_3.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model_3


In [26]:
densenet_model = build_densenet()
history_densenet = densenet_model.fit(train_generator, epochs=20, validation_data=val_generator)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 83s 585ms/step - accuracy: 0.2256 - loss: 2.4684 - val_accuracy: 0.3872 - val_loss: 1.6088
Epoch 2/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 36s 341ms/step - accuracy: 0.4533 - loss: 1.6132 - val_accuracy: 0.5827 - val_loss: 1.1856
Epoch 3/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 332ms/step - accuracy: 0.5296 - loss: 1.2880 - val_accuracy: 0.6741 - val_loss: 0.9578
Epoch 4/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 34s 327ms/step - accuracy: 0.5913 - loss: 1.1319 - val_accuracy: 0.7189 - val_loss: 0.8141
Epoch 5/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 333ms/step - accuracy: 0.6150 - loss: 1.0711 - val_accuracy: 0.7646 - val_loss: 0.7023
Epoch 6/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 341ms/step - accuracy: 0.6561 - loss: 0.9574 - val_accuracy: 0.7918 - val_loss: 0.6166
Epoch 7/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 337ms/step - accuracy: 0.6915 - loss: 0.8602 - val_accuracy: 0.8045 - val_loss: 0.5552
Epoch 8/20
97/97 ━━━━━━━━━━━━━━━━━━━━ 35s 337m

# This is the model with the highest test accuracy --> DenseNet121

In [31]:
test_loss, test_acc = densenet_model.evaluate(test_generator)
print(f"EfficientNetB3 Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.9430 - loss: 0.1990
EfficientNetB3 Test Accuracy: 0.9368, Test Loss: 0.2081


In [27]:
efficientnet_model.save("efficientnet_model_2.h5")
densenet_model.save("densenet_model_2.h5")